In [ ]:
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import talib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import math 


In [ ]:
path = "/Users/zed/AI_Lab/DoubleEnsembleML/Data/"
list = ["BTC","ETC",'ETH','FIL','LTC','XRP']
df = pd.read_csv(path +list[1]+".csv")

In [ ]:
datas = [0]*6

In [ ]:
for i in range(6):
    datas[i] = pd.read_csv(path +list[i]+".csv")
    if i==0:
        datas[i] = datas[i].loc[:,['Date','High','Low','Close','Volume (BTC)']].rename(columns = {'Volume (BTC)':'Volume'})
        
    if i>0:
        datas[i] = datas[i].rename(columns={'Last':'Close'})
        datas[i] = datas[i].drop(['Mid','Bid','Ask'],axis=1)
    datas[i]['Date'] = pd.to_datetime(datas[i]['Date'])
    datas[i] = datas[i].set_index('Date')

In [ ]:
for i in range(6):
    print(datas[i].columns)

In [ ]:
datas[0]

In [ ]:
fig,ax = plt.subplots(2,3,figsize = (20,8),dpi = 100)
for i in range(2):
    for j in range(3):
        df = datas[i+j]
        # ax[i,j].set_xticklabels(rotation=45)
        ax[i,j].plot(df['Close'])
        

In [ ]:
def cal_factor(dataset):
    dataset['Open'] = dataset['Close'].shift(-1)
    dataset['H-L'] = dataset['High'] - dataset['Low']
    # dataset['O-C'] = dataset['Close'] - dataset['Open']
    dataset['3day MA'] = dataset['Close'].shift(1).rolling(window = 3).mean()
    dataset['10day MA'] = dataset['Close'].shift(1).rolling(window = 10).mean()
    dataset['30day MA'] = dataset['Close'].shift(1).rolling(window = 30).mean()
    dataset['Std_dev']= dataset['Close'].rolling(5).std()
    dataset['RSI'] = talib.RSI(dataset['Close'].values, timeperiod = 9)
    dataset['ATR'] = talib.ATR(dataset['High'].values, dataset['Low'].values, dataset['Close'].values, 7)
    dataset['Upperband'], dataset['Middleband'], dataset['Lowerband'] = talib.BBANDS(dataset['Close'].values, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    # dataset['ADX'] = talib.ADX(dataset.High, dataset.Low, dataset.Close, timeperiod=14)
    for i in range(5):
        s = "Last "+str(i+1)+" Price"
        dataset[s] = dataset['Close'].shift(i+1)
    dataset['Target_value'] = (dataset.Close.shift(-1)-dataset.Close)/dataset.Close
    dataset['Target_tag'] = dataset.Target_value.apply(lambda x:1 if x>0 else 0)
    data  = dataset.dropna(how = "any")
    
    return data 
def sign(x):
    if x>0:
        return 1
    else: 
        return 0

In [ ]:
datas_fa = [0]*6
for i in range(6):
    datas_fa[i] = cal_factor(cal_factor(datas[i]))
    datas_fa[i].to_csv('./data_factor/'+list[i]+'_fac.csv')

In [ ]:
datas_fa[0]